In [1]:
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import roc_curve

from config import get_config
from models.swin_transformer import SwinTransformer
from main import build_model
import json
import math
import os
from sklearn.metrics import confusion_matrix
import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import gc
from functools import partial
from sklearn import metrics
from collections import Counter
import json
import itertools
from tensorflow.python.keras import backend
from config import get_config

ModuleNotFoundError: No module named 'yacs'

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [1]:
config = get_config(r"./configs/swin_base_patch4_window7_224.yaml")
model = build_model(config)
path = r"D:\project\neck region transformer\result\2021.6.16\swin_removemarkcross_randomcrop_batchsize8_lr0.001_SGD_imgsize224_epoch200_1fold.pth"
model_dict = model.state_dict()
params = torch.load(path,map_location=torch.device('cpu'))
new_state_dict = OrderedDict()
# 修改 key
for k, v in params['model'].items() :
    if 'head' not in k and 'patch_embed.proj.weight' not in k :
        new_state_dict[k] = v
model_dict.update(new_state_dict)
model.load_state_dict(model_dict)
transform_test = transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        # Normalize a tensor image with mean and standard variance
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

testset = torchvision.datasets.ImageFolder(root=r'D:\data\neck region src\test\k'+str(1), transform=transform_test) #/data/lyt/neck region src/test,D:\data\neck region src\test
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)
lossFunction = nn.CrossEntropyLoss()
lr = 0.001
base_optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
# optimizer = SAM(model.parameters(), base_optimizer, lr=lr, momentum=0.9)# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingWarmRestarts(base_optimizer, T_0=40, T_mult=2)

model.eval()  # enter test mode
test_loss = 0  # accumulate every batch loss in a epoch
correct = 0
total = 0
outputs=None
with torch.no_grad():

    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        print(outputs.size)
        loss = lossFunction(outputs, targets)  # compute loss

        test_loss += loss.item()  # accumulate every batch loss in a epoch
        _, predicted = outputs.max(1)  # make prediction according to the outputs
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item() 

        


# cm = confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(Y_pred, axis=1))

# cm_plot_label =['benign', 'malignant']
# plot_confusion_matrix(cm, cm_plot_label, title ='Confusion Metrix for Skin Cancer')

NameError: name 'get_config' is not defined